In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import KeyedVectors

Build the feed-forward neural network model:

In [3]:
class Net(nn.Module):
  def __init__(self, num_words, emb_dim, num_y, embeds):
    super().__init__()
    self.emb = nn.Embedding(num_words, emb_dim)
    self.emb.weight = nn.Parameter(torch.Tensor(embeds))
    self.linear = nn.Linear(emb_dim, num_y)
    self.sigmoid = nn.Sigmoid()

  def forward(self, text):
    embeds = torch.mean(self.emb(text), dim=0)
    return self.sigmoid(self.linear(embeds))

In [4]:
def load_vocab(text):
  word_to_ix = {}
  for sent , label in text:
    for word in sent.split():
      word_to_ix.setdefault(word, len(word_to_ix))
  return word_to_ix

Define sample train and test sentences with labels (0 neg/1 pos):

In [5]:
train_data = [("I'm so happy", 1), ("I'm so sad", 0)]
tok_to_ix = load_vocab(train_data)

Create a new model and set the optimizer and loss function:

In [8]:
emb_dim = 300
num_classes = 1
learning_rate = 0.01
embeds = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary = True).wv.vectors
model = Net(len(tok_to_ix), emb_dim, num_classes, embeds)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

/opt/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


Train the model for however many epochs you want!

In [9]:
n_epochs = 5
for epoch in range(n_epochs):
  model.train()
  for text, label in train_data:
    x = [tok_to_ix[tok] for tok in text.split()]
    x_train_tensor = torch.LongTensor(x)
    y_train_tensor = torch.Tensor([label])
    pred_y = model(x_train_tensor)
    loss = loss_fn(pred_y, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("\nEpoch:", epoch)
  print("Training loss:", loss.item())

KeyboardInterrupt: 

Last step! Evaluate the model to see if it predicts the right label:

In [ ]:
y_test = 'sad face'
with torch.no_grad():
  model.eval()
  x = [tok_to_ix[tok] for tok in y_test.split() if tok in tok_to_ix]
  x_test = torch.LongTensor(x)
  pred_y_test = model(x_test)

  print('Test sentence:', y_test)
  print('Predicted label:', pred_y_test.numpy())

Test sentence: sad face
Predicted label: [0.4775631]
